In [1]:
"""Running a global fund

Demonstration of using LUSID to run funds fed from multiple source systems across multiple regions

Attributes
----------
instruments
transactions
cut labels
aggregation
recipes
quotes
cocoon
transaction configuration
"""

'Running a global fund\n\nDemonstration of using LUSID to run funds fed from multiple source systems across multiple regions\n\nAttributes\n----------\ninstruments\ntransactions\ncut labels\naggregation\nrecipes\nquotes\ncocoon\ntransaction configuration\n'

## 0) Import Libraries and Initialise LUSID Client

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools
import lusidtools.cocoon.cocoon as cocoon_tools
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
globals = {}
                            
# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")
    
print('LUSID Client Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Client Initialised
LUSID version :  0.6.10068.0


## 1) Load Instrument Master across Regions

### Fetch the Instrument Master Data

#### US Instrument Master

In [3]:
us_instrument_master = pd.read_csv("data/global-fund-US-instrument-master.csv")

us_instrument_master.head(n=20)

,isocur,moodys_rating,name,isin,figi,assettype,issuercountry,bloombergticker,exchange_traded,couprate,int_id,sector
0,USD,NaN,Amazon_Nasdaq_AMZN,US0231351067,BBG000BVPXP1,common_stock,united_states_america,AMZN,UN,NaN,imd_34634534,equity
1,USD,NaN,Apple_Nasdaq_AAPL,US0378331005,BBG000B9XVV8,common_stock,united_states_america,AAPL,UN,NaN,imd_35345345,equity
2,USD,Aa2,USTreasury_2.00_2021,US912828U816,BBG00FN3B5K8,us_government,united_states_america,T 2 12/31/21,BERLIN,2.00,imd_34535347,govt
3,USD,Aa2,USTreasury_6.875_2025,US912810EV62,BBG000DQQNJ8,us_government,united_states_america,T 6.875 08/15/25,NEW YORK,6.88,imd_34534539,govt
4,USD,Baa1,TESCO_6.15_2037,US881575AC87,NaN,corporate_bond,united_states_america,NaN,NaN,6.15,imd_54001121,corporate


#### UK Instrument Master

In [4]:
uk_instrument_master = pd.read_csv("data/global-fund-UK-instrument-master.csv")

uk_instrument_master.head(n=20)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon,s&p rating
0,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN,NaN
1,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN,NaN
2,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN,NaN
3,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,NaN,NaN
4,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,NaN,NaN
5,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,NaN,NaN
6,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,NaN,NaN
7,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,NaN,NaN
8,Sage_LondonStockEx_SGE,imd_23536673,GBP,GB00B8C3BL03,BBG000BN0PP3,LN,united_kingdom,SGE,equity,common_stock,NaN,NaN
9,TESCO_LondonStockEx_TSCO,imd_34634673,GBP,GB0008847096,BBG000BF46Y8,LN,united_kingdom,TSCO,equity,common_stock,NaN,NaN


#### Combined Instrument Master

In [5]:
combined_instrument_master = pd.read_csv("data/global-fund-combined-instrument-master.csv")

combined_instrument_master.head(n=30)

,instrument_name,client_internal,currency,isin,figi,couprate,s&p rating,moodys_rating
0,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,NaN,NaN,NaN
1,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,NaN,NaN,NaN
2,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,NaN,NaN,NaN
3,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,NaN,NaN,NaN
4,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,NaN,NaN,NaN
5,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,NaN,NaN,NaN
6,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,NaN,NaN,NaN
7,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,NaN,NaN,NaN
8,Sage_LondonStockEx_SGE,imd_23536673,GBP,GB00B8C3BL03,BBG000BN0PP3,NaN,NaN,NaN
9,TESCO_LondonStockEx_TSCO,imd_34634673,GBP,GB0008847096,BBG000BF46Y8,NaN,NaN,NaN


### Load the Instruments into LUSID

In [6]:
instrument_properties_scope = 'InstrumentProperties005'

instrument_identifier_mapping = {
    "Figi": "figi",
    "Isin": "isin",
    "ClientInternal": "client_internal"
}

instrument_mapping_required = {
    "name": "instrument_name"
}

instrument_mapping_optional = {}

responses = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    scope=instrument_properties_scope, 
    data_frame=combined_instrument_master, 
    mapping_required=instrument_mapping_required, 
    mapping_optional=instrument_mapping_optional, 
    file_type="instrument", 
    identifier_mapping=instrument_identifier_mapping, 
    property_columns=["s&p rating", "moodys_rating", "currency"])

prettyprint.instrument_response(responses["instruments"]["success"][0])

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,ClientInternal: imd_34001121,imd_34001121,LUID_WNY0FCEW
1,ClientInternal: imd_34567338,imd_34567338,LUID_0000MIP9
2,ClientInternal: imd_35349900,imd_35349900,LUID_0000MIP2
3,ClientInternal: imd_34534539,imd_34534539,LUID_0000MIP1
4,ClientInternal: imd_34534534,imd_34534534,LUID_0000MIPA
5,ClientInternal: imd_43535553,imd_43535553,LUID_0003UX1P
6,ClientInternal: imd_34535347,imd_34535347,LUID_0000MIPE
7,ClientInternal: imd_34534536,imd_34534536,LUID_0000MIP3
8,ClientInternal: imd_34634673,imd_34634673,LUID_0000NA4Q
9,ClientInternal: imd_34535552,imd_34535552,LUID_0000MIOZ


## 2) Load Transactions

### Create Transaction Portfolios

In [7]:
scopes = {
    "UK": "UK_Thinkfolio",
    "US": "US_SimcorpDimension"
}

responses = global_fund_tools.create_portfolios(
    api_factory=api_factory, 
    scopes=list(scopes.values()), 
    code="GlobalCreditFund", 
    currency="EUR")

for response in responses:
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_Thinkfolio
Code: GlobalCreditFund
Portfolio Effective From: 2009-01-24 10:45:46.581263+00:00
Portfolio Created On: 2022-10-03 10:45:47.211611+00:00

Portfolio Created
Scope: US_SimcorpDimension
Code: GlobalCreditFund
Portfolio Effective From: 2009-01-24 10:45:46.581263+00:00
Portfolio Created On: 2022-10-03 10:45:47.911222+00:00



### Fetch the Transaction Data

#### US

In [8]:
us_transactions = pd.read_csv("data/global-fund-us-transactions.csv")

us_transactions.head(n=20)

,portfolio_code,transaction_id,instrument_name,transaction_description,transaction_type,transaction_units,transaction_price,transaction_currency,company_exposure,transaction_cost,figi,transaction_date,settlement_date,isin,mtom,accmethod,broker,region,client_internal
0,GlobalCreditFund,ab1543673,USD,Initial Funds In,FundsIn,1000000000,1,USD,NaN,1000000000,NaN,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,NaN,0,13,xy35920,CR,NaN
1,GlobalCreditFund,xfg352235,USTreasury_2.00_2021,Purchase of US Treasury Bonds,BY,50000,99,USD,USGovt,4950000,BBG00FN3B5K8,2019-09-03T18:42:12.435000+00:00,2019-09-05T18:42:12.435000+00:00,US912828U816,1,14,xy90994,CR,imd_34535347
2,GlobalCreditFund,ccv335235,USTreasury_6.875_2025,Purchase of US Treasury Bonds,BY,130000,127,USD,USGovt,16510000,BBG000DQQNJ8,2019-09-06T19:12:44.090000+00:00,2019-09-08T19:12:44.090000+00:00,US912810EV62,0,15,xy90994,CR,imd_34534539
3,GlobalCreditFund,cvcg767j31,TESCO_6.15_2037,Purchase of Tesco Corporate Bonds,BY,1750000,99,USD,TESCO,173250000,NaN,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,US881575AC87,0,21,xy90994,CR,imd_54001121


#### UK

In [9]:
uk_transactions = pd.read_csv("data/global-fund-uk-transactions.csv")

uk_transactions.head(n=20)

,desc,qty,unit_price,trade_currency,exposure,total_trade_amount,figi_identifier,trade_datetime,settlement_datetime,isin_identifier,compls,executor,location,asset_name,val,fund_code,t_id,internal_identifier
0,StartingBalance,2500000000,1,GBP,NaN,2500000000,NaN,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,NaN,MERC,jx112421,"Dublin, Ireland",GBP,532352352.00,GlobalCreditFund,kjk99898990z88908,NaN
1,Purchase,250000,149,GBP,UKGovt,37250000,BBG006N6HZM7,2019-09-03T14:42:12.435000+00:00,2019-09-05T14:42:12.435000+00:00,GB00BN65R313,SFP,jx112422,"Dublin, Ireland",UKGiltTreasury_3.5_2045,2335240000000.00,GlobalCreditFund,435opkplk03sfsf33,imd_54234532
2,Purchase,1500000,104,GBP,UKGovt,156000000,BBG001KKJLR4,2019-09-05T15:12:44.090000+00:00,2019-09-07T15:12:44.090000+00:00,GB00B4RMG977,LLK,jx112423,"Dublin, Ireland",UKGiltTreasury_3.75_2021,25252.00,GlobalCreditFund,dsfsdlkjs933234211,imd_34643653
3,Purchase,2200000,154,GBP,UKGovt,338800000,BBG0000D14P3,2019-09-10T09:58:43.112000+00:00,2019-09-11T09:58:43.112000+00:00,GB00B52WS153,LLK,jp392431,"Dublin, Ireland",UKGiltTreasury_4.5_2034,222532.00,GlobalCreditFund,dfklsjd90232350239,imd_34534534
4,Purchase,190000,108,GBP,TESCO,20520000,NaN,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,XS0414345974,MLLK,jp392431,"Luxembourg, Luxembourg",TESCO_6.13_2022,235235232.00,GlobalCreditFund,kj0llk90f0909201141,imd_34001121


#### Combined

In [10]:
combined_transactions = pd.read_csv("data/global-fund-combined-transactions.csv")

for transaction_type in ["StartingBalance", "FundsIn"]:
    combined_transactions.loc[
        combined_transactions[
            "transaction_type"] == transaction_type, 'currency_transaction'] = combined_transactions["trade_currency"]
    
combined_transactions["exchange_rate"] = 1

combined_transactions.head(n=30)

,portfolio_code,id,transaction_type,transaction_date,settlement_date,units,transaction_price,amount,trade_currency,instrument_name,...,location_region,isin,figi,client_internal,exposure_counterparty,val,compls,source,currency_transaction,exchange_rate
0,GlobalCreditFund,ab1543673,FundsIn,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,1000000000,1,1000000000,USD,USD,...,CR,NaN,NaN,NaN,NaN,NaN,NaN,US_SimcorpDimension,USD,1
1,GlobalCreditFund,xfg352235,BY,2019-09-03T18:42:12.435000+00:00,2019-09-05T18:42:12.435000+00:00,50000,99,4950000,USD,USTreasury_2.00_2021,...,CR,BBG00FN3B5K8,US912828U816,imd_34535347,USGovt,NaN,NaN,US_SimcorpDimension,NaN,1
2,GlobalCreditFund,ccv335235,BY,2019-09-06T19:12:44.090000+00:00,2019-09-08T19:12:44.090000+00:00,130000,127,16510000,USD,USTreasury_6.875_2025,...,CR,BBG000DQQNJ8,US912810EV62,imd_34534539,USGovt,NaN,NaN,US_SimcorpDimension,NaN,1
3,GlobalCreditFund,cvcg767j31,BY,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,1750000,99,173250000,USD,TESCO_6.15_2037,...,CR,NaN,US881575AC87,imd_54001121,TESCO,NaN,NaN,US_SimcorpDimension,NaN,1
4,GlobalCreditFund,kjk99898990z88908,StartingBalance,2019-09-01T09:31:22.664000+00:00,2019-09-01T09:31:22.664000+00:00,2500000000,1,2500000000,GBP,NaN,...,"Dublin, Ireland",NaN,NaN,NaN,NaN,532352352.00,MERC,UK_Thinkfolio,GBP,1
5,GlobalCreditFund,435opkplk03sfsf33,Purchase,2019-09-03T14:42:12.435000+00:00,2019-09-05T14:42:12.435000+00:00,250000,149,37250000,GBP,NaN,...,"Dublin, Ireland",GB00BN65R313,BBG006N6HZM7,imd_54234532,UKGovt,2335240000000.00,SFP,UK_Thinkfolio,NaN,1
6,GlobalCreditFund,dsfsdlkjs933234211,Purchase,2019-09-05T15:12:44.090000+00:00,2019-09-07T15:12:44.090000+00:00,1500000,104,156000000,GBP,NaN,...,"Dublin, Ireland",GB00B4RMG977,BBG001KKJLR4,imd_34643653,UKGovt,25252.00,LLK,UK_Thinkfolio,NaN,1
7,GlobalCreditFund,dfklsjd90232350239,Purchase,2019-09-10T09:58:43.112000+00:00,2019-09-11T09:58:43.112000+00:00,2200000,154,338800000,GBP,NaN,...,"Dublin, Ireland",GB00B52WS153,BBG0000D14P3,imd_34534534,UKGovt,222532.00,LLK,UK_Thinkfolio,NaN,1
8,GlobalCreditFund,kj0llk90f0909201141,Purchase,2019-09-10T13:58:57.113000+00:00,2019-09-11T13:58:57.113000+00:00,190000,108,20520000,GBP,NaN,...,"Luxembourg, Luxembourg",XS0414345974,NaN,imd_34001121,TESCO,235235232.00,MLLK,UK_Thinkfolio,NaN,1


#### Load the Transactions into LUSID

In [11]:
transaction_field_mapping_required = {
    "code": "portfolio_code",
    "transaction_id": "id",
    "type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "units",
    "transaction_price.price": "transaction_price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "amount",
    "total_consideration.currency": "trade_currency",
    "transaction_currency": "trade_currency"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate"
}

transaction_identifier_mapping = {
    "Figi": "figi",
    "Isin": "isin",
    "ClientInternal": "client_internal",
    "Currency": "currency_transaction"
}

for scope in list(scopes.values()):
    
    transaction_field_mapping_optional["source"] = f"${scope}"

    responses = cocoon_tools.load_from_data_frame(
        api_factory=api_factory, 
        scope=scope, 
        data_frame=combined_transactions.loc[
            combined_transactions["source"] == scope],
        mapping_required=transaction_field_mapping_required,
        mapping_optional=transaction_field_mapping_optional,
        identifier_mapping=transaction_identifier_mapping,
        file_type='transaction',
        property_columns=[
            "instrument_name", 
            "accounting_method",
            "mtom",
            "broker_executor",
            "location_region",
            "exposure_counterparty",
            "val",
            "compls",
            ])

    for response in responses['transactions']['success']:
        prettyprint.transactions_response(response, scope, response.href.split('/')[7])

Transactions Successfully Upserted into Portfolio
Scope: UK_Thinkfolio
Code: GlobalCreditFund
Transactions Effective From: 2019-09-10 13:58:57.113000+00:00
Transactions Created On: 2022-10-03 10:45:57.029202+00:00

Transactions Successfully Upserted into Portfolio
Scope: US_SimcorpDimension
Code: GlobalCreditFund
Transactions Effective From: 2019-09-10 13:58:57.113000+00:00
Transactions Created On: 2022-10-03 10:45:59.915173+00:00



## 3) Configure Transaction Types

### Get Unique Types

In [12]:
combined_transactions.groupby(["transaction_type", "source"]).agg({"units": "max"})

,,units
transaction_type,source,
BY,US_SimcorpDimension,1750000
FundsIn,US_SimcorpDimension,1000000000
Purchase,UK_Thinkfolio,2200000
StartingBalance,UK_Thinkfolio,2500000000


### Create Configuration for Each Type

In [13]:
movements1=[
    models.TransactionTypeMovement(
        movement_types='StockMovement',
        side='Side1',
        direction=1,
        properties=None,
        mappings=None),
    models.TransactionTypeMovement(
        movement_types='CashCommitment',
        side='Side2',
        direction=-1,
        properties=None,
        mappings=None)
]

response = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="US_SimcorpDimension",
    type="BY",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="BY",
                description="BY",
                transaction_class="BY",
                transaction_roles='None',
            )
        ],
        movements=movements1,
    )
)

prettyprint.transaction_type_response(response)

response = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="UK_Thinkfolio",
    type="Purchase",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="Purchase",
                description="Purchase",
                transaction_class="Purchase",
                transaction_roles='None',
            )
        ],
        movements=movements1,
    )
)
prettyprint.transaction_type_response(response)

movements2=[
    models.TransactionTypeMovement(
        movement_types='CashCommitment',
        side='Side2',
        direction=1,
        properties=None,
        mappings=None)
]

response = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="US_SimcorpDimension",
    type="FundsIn",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="FundsIn",
                description="FundsIn",
                transaction_class="FundsIn",
                transaction_roles='None',
            )
        ],
        movements=movements2,
    )
)

prettyprint.transaction_type_response(response)

response = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="UK_Thinkfolio",
    type="StartingBalance",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="StartingBalance",
                description="StartingBalance",
                transaction_class="StartingBalance",
                transaction_roles='None',
            )
        ],
        movements=movements2,
    )
)
prettyprint.transaction_type_response(response)

Transaction Configuration #61

Transaction Type Aliases
Transaction Type: BY
Alias Description: PURCHASE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #79

Transaction Type Aliases
Transaction Type: BY
Alias Description: BY
Transaction Class: BY
Transaction Group: US_SimcorpDimension
Transaction Roles: AllRoles


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #81

Transaction Type Aliases
Transaction Type: Purchase
Alias Description: Purchase
Transaction Class: Purchase
Transaction Group: UK_Thinkfolio
Transaction Roles: AllRoles


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction

## 4) Create Cut Labels for Region Closes

In [14]:
responses = global_fund_tools.create_cut_labels(
    api_factory=api_factory,
    exchange_names=["LSE", "NYSE"], 
    cut_label_type="market_close")

for response in responses:
    prettyprint.cut_label_response(response)

Cut Label Created
Display Name: LSE_market_close
Code: LSE_market_close
Local Time: 16:30
Timezone: GB
Description: LSE_market_close which is at 16:30 local time

Cut Label Created
Display Name: NYSE_market_close
Code: NYSE_market_close
Local Time: 16:00
Timezone: America/New_York
Description: NYSE_market_close which is at 16:0 local time



## 5) Load Close of Day Prices

In [15]:
marketdata = pd.read_csv("data/global-fund-marketdata.csv")

marketdata['time'] = marketdata['time'].apply(lambda x: "LSE_market_close" if x=="LSEClose" else "NYSE_market_close")
marketdata['date_cutlabel'] = marketdata['date'] + 'N' + marketdata['time']

marketdata.head(n=30)

,figi,isin,client_internal,name,currency,date,time,price,source,type,date_cutlabel
0,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,2019-09-11,LSE_market_close,106.51,bloomberg,mid_price,2019-09-11NLSE_market_close
1,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,2019-09-10,LSE_market_close,106.59,bloomberg,mid_price,2019-09-10NLSE_market_close
2,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,2019-09-11,LSE_market_close,106.52,reuters,mid_price,2019-09-11NLSE_market_close
3,BBG001KKJLR4,GB00B4RMG977,imd_34643653,UKGiltTreasury_3.75_2021,GBP,2019-09-10,LSE_market_close,106.59,reuters,mid_price,2019-09-10NLSE_market_close
4,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,2019-09-11,LSE_market_close,151.44,bloomberg,mid_price,2019-09-11NLSE_market_close
5,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,2019-09-10,LSE_market_close,152.28,bloomberg,mid_price,2019-09-10NLSE_market_close
6,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,2019-09-11,LSE_market_close,151.22,reuters,mid_price,2019-09-11NLSE_market_close
7,BBG0000D14P3,GB00B52WS153,imd_34534534,UKGiltTreasury_4.5_2034,GBP,2019-09-10,LSE_market_close,152.29,reuters,mid_price,2019-09-10NLSE_market_close
8,BBG006N6HZM7,GB00BN65R313,imd_54234532,UKGiltTreasury_3.5_2045,GBP,2019-09-11,LSE_market_close,152.44,bloomberg,mid_price,2019-09-11NLSE_market_close
9,BBG006N6HZM7,GB00BN65R313,imd_54234532,UKGiltTreasury_3.5_2045,GBP,2019-09-10,LSE_market_close,153.16,bloomberg,mid_price,2019-09-10NLSE_market_close


In [16]:
marketdata_scopes = {
    'bloomberg': 'MarketData_Bloomberg_123',
    'reuters': 'MarketData_Reuters_123'
}

instrument_identifier_mapping = {
    'identifier_mapping': {
        "Figi": "figi",
        "Isin": "isin",
        "CurrencyPair": "currency"
    }
}

instrument_identifier_heirarchy = ["Figi", "Isin", "CurrencyPair"]

quotes_mapping_required = {
    "quote_type": "type",
    "effective_at": "date_cutlabel",
    "currency": "currency",
    "value": "price"
}
    
response_1 = global_fund_tools.upsert_quotes(
    api_factory=api_factory,
    scope=marketdata_scopes['bloomberg'],
    data_frame=marketdata.loc[marketdata['source'] == 'bloomberg'],
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_identifier_heirarchy=instrument_identifier_heirarchy, 
    required_mapping=quotes_mapping_required)


response_2 = global_fund_tools.upsert_quotes(
    api_factory=api_factory,
    scope=marketdata_scopes['reuters'],
    data_frame=marketdata.loc[marketdata['source'] == 'reuters'],
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_identifier_heirarchy=instrument_identifier_heirarchy, 
    required_mapping=quotes_mapping_required)


pd.concat([response_1, response_2], ignore_index=True)

,local_vars_configuration,_lineage,_cut_label,_uploaded_by,_as_at,_scale_factor,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-11NLSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG001KKJLR4,Figi,Price,Mid,106.51,GBP,Success
1,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-10NLSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG001KKJLR4,Figi,Price,Mid,106.59,GBP,Success
2,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-11NLSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG0000D14P3,Figi,Price,Mid,151.44,GBP,Success
3,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-10NLSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG0000D14P3,Figi,Price,Mid,152.28,GBP,Success
4,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-11NLSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG006N6HZM7,Figi,Price,Mid,152.44,GBP,Success
5,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-10NLSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG006N6HZM7,Figi,Price,Mid,153.16,GBP,Success
6,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-11NNYSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,US881575AC87,Isin,Price,Mid,97.00,USD,Success
7,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-10NNYSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,US881575AC87,Isin,Price,Mid,96.90,USD,Success
8,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-11NNYSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG000DQQNJ8,Figi,Price,Mid,129.55,USD,Success
9,<lusid.configuration.Configuration object at 0...,InternalSystem,2019-09-10NNYSE_market_close,00ubsbczibzRxZJzM2p7,2022-10-03 10:46:06.137264+00:00,None,None,DataScope,None,BBG000DQQNJ8,Figi,Price,Mid,129.22,USD,Success


## 6) Group Portfolios from the US & UK

In [17]:
portfolio_group_scope = "Worldwide"

response = global_fund_tools.create_portfolio_group(
    api_factory=api_factory, 
    scope=portfolio_group_scope, 
    code="GlobalCreditFund", 
    portfolios=[
        models.ResourceId(
            scope=scopes["US"],
            code="GlobalCreditFund"),
        models.ResourceId(
            scope=scopes["UK"],
            code="GlobalCreditFund")
    ])

prettyprint.portfolio_group_response(response, "created")

Portfolio Group Created
Name: GlobalCreditFund
Scope: Worldwide
Code: GlobalCreditFund
Portfolios Inside Group: 
GlobalCreditFund
GlobalCreditFund




## 7) Value the GlobalCreditFund for each Region Close

### LSE Close 11th September - Bloomberg Market Data

In [18]:
global_fund_tools.valuation(
    api_factory=api_factory, 
    marketdata_scope=marketdata_scopes['bloomberg'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NLSE_market_close")

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Valuation/PvInReportCcy),Proportion(Valuation/PvInReportCcy),currency
0,CCY_GBP,GBP,"1,947,430,000.00","1,947,430,000.00","2,181,121,600.00",0.00,EUR
1,LUID_0000MIPC,UKGiltTreasury_3.5_2045,"250,000.00","37,250,000.00","520,100.00",0.00,EUR
2,LUID_0000MIP6,UKGiltTreasury_3.75_2021,"1,500,000.00","156,000,000.00","178,936,800.00",0.00,EUR
3,LUID_0000MIPA,UKGiltTreasury_4.5_2034,"2,200,000.00","338,800,000.00","4,016,320.00",0.00,EUR
4,LUID_WNY0FCEW,TESCO_6.13_2022,"190,000.00","20,520,000.00","23,408,000.00",0.00,EUR
5,CCY_USD,USD,"805,290,000.00","805,290,000.00","732,813,900.00",0.00,EUR
6,LUID_0000MIPE,USTreasury_2.00_2021,"50,000.00","4,950,000.00","4,585,945.00",0.00,EUR
7,LUID_0000MIP1,USTreasury_6.875_2025,"130,000.00","16,510,000.00","105,083,918,333,333.33",1.00,EUR
8,LUID_Z4J1L8PW,TESCO_6.15_2037,"1,750,000.00","173,250,000.00","154,313,250.00",0.00,EUR
9,NaN,NaN,"2,758,790,000.00","3,500,000,000.00","105,087,198,049,248.33",1.00,NaN


### LSE Close 11th September - Reuters Market Data

In [19]:
global_fund_tools.valuation(
    api_factory=api_factory, 
    marketdata_scope=marketdata_scopes['reuters'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NLSE_market_close")

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Valuation/PvInReportCcy),Proportion(Valuation/PvInReportCcy),currency
0,CCY_USD,USD,"805,290,000.00","805,290,000.00","732,813,900.00",0.00,EUR
1,LUID_0000MIPE,USTreasury_2.00_2021,"50,000.00","4,950,000.00","4,585,945.00",0.00,EUR
2,LUID_0000MIP1,USTreasury_6.875_2025,"130,000.00","16,510,000.00","105,083,918,333,333.33",1.00,EUR
3,LUID_Z4J1L8PW,TESCO_6.15_2037,"1,750,000.00","173,250,000.00","157,466,400.00",0.00,EUR
4,CCY_GBP,GBP,"1,947,430,000.00","1,947,430,000.00","2,181,121,600.00",0.00,EUR
5,LUID_0000MIPC,UKGiltTreasury_3.5_2045,"250,000.00","37,250,000.00","520,100.00",0.00,EUR
6,LUID_0000MIP6,UKGiltTreasury_3.75_2021,"1,500,000.00","156,000,000.00","178,953,600.00",0.00,EUR
7,LUID_0000MIPA,UKGiltTreasury_4.5_2034,"2,200,000.00","338,800,000.00","4,016,320.00",0.00,EUR
8,LUID_WNY0FCEW,TESCO_6.13_2022,"190,000.00","20,520,000.00","23,876,160.00",0.00,EUR
9,NaN,NaN,"2,758,790,000.00","3,500,000,000.00","105,087,201,687,358.33",1.00,NaN


### NYSE Close 11th September - Reuters Market Data

In [20]:
global_fund_tools.valuation(
    api_factory=api_factory, 
    marketdata_scope=marketdata_scopes['reuters'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NNYSE_market_close")

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Valuation/PvInReportCcy),Proportion(Valuation/PvInReportCcy),currency
0,CCY_GBP,GBP,"1,947,430,000.00","1,947,430,000.00","2,181,121,600.00",0.00,EUR
1,LUID_0000MIPC,UKGiltTreasury_3.5_2045,"250,000.00","37,250,000.00","520,100.00",0.00,EUR
2,LUID_0000MIP6,UKGiltTreasury_3.75_2021,"1,500,000.00","156,000,000.00","178,953,600.00",0.00,EUR
3,LUID_0000MIPA,UKGiltTreasury_4.5_2034,"2,200,000.00","338,800,000.00","4,016,320.00",0.00,EUR
4,LUID_WNY0FCEW,TESCO_6.13_2022,"190,000.00","20,520,000.00","23,876,160.00",0.00,EUR
5,CCY_USD,USD,"805,290,000.00","805,290,000.00","732,813,900.00",0.00,EUR
6,LUID_0000MIPE,USTreasury_2.00_2021,"50,000.00","4,950,000.00","4,585,490.00",0.00,EUR
7,LUID_0000MIP1,USTreasury_6.875_2025,"130,000.00","16,510,000.00","105,083,918,333,333.33",1.00,EUR
8,LUID_Z4J1L8PW,TESCO_6.15_2037,"1,750,000.00","173,250,000.00","157,657,500.00",0.00,EUR
9,NaN,NaN,"2,758,790,000.00","3,500,000,000.00","105,087,201,878,003.33",1.00,NaN


### NYSE Close 11th September - Bloomberg Market Data

In [21]:
global_fund_tools.valuation(
    api_factory=api_factory, 
    marketdata_scope=marketdata_scopes['bloomberg'], 
    portfolio_group=models.ResourceId(
        scope=portfolio_group_scope,
        code="GlobalCreditFund"),
    time="2019-09-11NNYSE_market_close")

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Valuation/PvInReportCcy),Proportion(Valuation/PvInReportCcy),currency
0,CCY_USD,USD,"805,290,000.00","805,290,000.00","732,813,900.00",0.00,EUR
1,LUID_0000MIPE,USTreasury_2.00_2021,"50,000.00","4,950,000.00","4,585,035.00",0.00,EUR
2,LUID_0000MIP1,USTreasury_6.875_2025,"130,000.00","16,510,000.00","105,083,918,333,333.33",1.00,EUR
3,LUID_Z4J1L8PW,TESCO_6.15_2037,"1,750,000.00","173,250,000.00","154,472,500.00",0.00,EUR
4,CCY_GBP,GBP,"1,947,430,000.00","1,947,430,000.00","2,181,121,600.00",0.00,EUR
5,LUID_0000MIPC,UKGiltTreasury_3.5_2045,"250,000.00","37,250,000.00","520,100.00",0.00,EUR
6,LUID_0000MIP6,UKGiltTreasury_3.75_2021,"1,500,000.00","156,000,000.00","178,936,800.00",0.00,EUR
7,LUID_0000MIPA,UKGiltTreasury_4.5_2034,"2,200,000.00","338,800,000.00","4,016,320.00",0.00,EUR
8,LUID_WNY0FCEW,TESCO_6.13_2022,"190,000.00","20,520,000.00","23,408,000.00",0.00,EUR
9,NaN,NaN,"2,758,790,000.00","3,500,000,000.00","105,087,198,207,588.33",1.00,NaN
